In [37]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [38]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

In [39]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [40]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [41]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [42]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [43]:
## 
n='Input_A2_016' # 改名稱
##
dire= ['./Data_Science_2020_imdb/BayesianRidge_Pre/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_A2_016
./Data_Science_2020_imdb/BayesianRidge_Pre/
feature number: 2
train_rmse: 0.008788277629346544
test_rmse: 0.009079627197977655
test_r2: 0.5088963969039266
feature number: 3
train_rmse: 0.008477487835452484
test_rmse: 0.008808154790360715
test_r2: 0.5365004908227714
feature number: 4
train_rmse: 0.008181302740127188
test_rmse: 0.008509857811724053
test_r2: 0.5690125990799021
feature number: 5
train_rmse: 0.008148505427006467
test_rmse: 0.008503435128398995
test_r2: 0.5697894905455707
feature number: 6
train_rmse: 0.008044798708574738
test_rmse: 0.008421744154343411
test_r2: 0.5779946965784082
feature number: 7
train_rmse: 0.008037700766365799
test_rmse: 0.008429081780223977
test_r2: 0.577175770795938
feature number: 8
train_rmse: 0.008028864729087523
test_rmse: 0.008428214455153963
test_r2: 0.5771949852630488
feature number: 9
train_rmse: 0.008021166376215437
test_rmse: 0.008470190176667615
test_r2: 0.5733351515250095
feature number: 10
train_rmse: 0.007989596729066649
test

train_rmse: 0.00791822831167259
test_rmse: 0.008778672328997917
test_r2: 0.5420149206170048
feature number: 17
train_rmse: 0.007892347393384336
test_rmse: 0.008761221554147675
test_r2: 0.5440381773611984
feature number: 18
train_rmse: 0.00788675741999125
test_rmse: 0.008791871511298592
test_r2: 0.540724611536539
feature number: 19
train_rmse: 0.00778676851068145
test_rmse: 0.008713815546174598
test_r2: 0.5486525510165771
feature number: 20
train_rmse: 0.007762302833102685
test_rmse: 0.008731681503967512
test_r2: 0.5471833715195537
Test_rmse_min = 1.fea_num: 6  2.rmse: 0.008421744154343411
Test_r2_max = 1. fea_num: 6  2.r2: 0.5779946965784082
./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.008788277629346544
test_rmse: 0.009079627197977655
test_r2: 0.5088963969039266
feature number: 3
train_rmse: 0.008477487835452484
test_rmse: 0.008808154790360715
test_r2: 0.5365004908227714
feature number: 4
train_rmse: 0.008181302740127188
test_rmse: 0.0085098578117

train_rmse: 0.008021166376215437
test_rmse: 0.008470190176667615
test_r2: 0.5733351515250095
feature number: 10
train_rmse: 0.007989596729066649
test_rmse: 0.008549104726173207
test_r2: 0.5650616362471967
feature number: 11
train_rmse: 0.007981193744971467
test_rmse: 0.008581680782726836
test_r2: 0.5618494272461202
feature number: 12
train_rmse: 0.007967289423163753
test_rmse: 0.008581704819460625
test_r2: 0.5618772374429635
feature number: 13
train_rmse: 0.007957278326445903
test_rmse: 0.00860407631349951
test_r2: 0.5599246762847139
feature number: 14
train_rmse: 0.007942331126945628
test_rmse: 0.00868762921178079
test_r2: 0.5510265006399556
feature number: 15
train_rmse: 0.00793544227215513
test_rmse: 0.008704434718082557
test_r2: 0.5493185688147577
feature number: 16
train_rmse: 0.00791822831167259
test_rmse: 0.008778672328997917
test_r2: 0.5420149206170048
feature number: 17
train_rmse: 0.007892347393384336
test_rmse: 0.008761221554147675
test_r2: 0.5440381773611984
feature number:

# 可以修改第一層的神經元數量

In [44]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(7, activation='relu',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [45]:
fea_num=[6,7,8,9] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/BayesianRidge_Pre/Input_A2_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 6
test_rmse_ave: 0.009761363439023498
[0.009680836061341609, 0.00869705407753889, 0.011659288969554548, 0.009244250369956979, 0.00825661623916505, 0.007973851543156212, 0.011777013277084884, 0.010046956504848792, 0.011022067269348736, 0.009255700078239287]


test_r2_ave: 0.41964743728153253
[0.5184608331787938, 0.5931811037311634, 0.13659862490525754, 0.4856424764847497, 0.605247482968939, 0.6315538334394413, 0.07747781968423462, 0.4055149728053564, 0.23634184266765013, 0.5064553829497394]


feature number: 7
test_rmse_ave: 0.010452298624676361
[0.009071846679261468, 0.015590061294236962, 0.009753228456328825, 0.009281052862375657, 0.007496315944014382, 0.009724580134585204, 0.010173521888310862, 0.010935787156175162, 0.009848470462972029, 0.012648121368503058]


test_r2_ave: 0.33498051984869576
[0.57713933440425, -0.3072312507044166, 0.3958213175094575, 0.4815388837205056, 0.6746008054495776, 0.4520013737608479, 0.3115868857495846, 0.2956768316520121, 0.390308059466312

In [ ]:
fea_num=[6,7,8,9]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/Input_A2_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[6,7,8,9] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/Input_A2_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[6,7,8,9]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/Input_A2_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[6,7,8,9] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/Input_A2_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[6,7,8,9] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/Input_A2_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[6,7,8,9] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/Input_A2_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[6,7,8,9] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/Input_A2_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')